In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy import stats
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.datasets import make_regression
import sklearn
from sklearn.model_selection import GridSearchCV
np.random.seed(10)
import lightgbm as lgb
from lightgbm import plot_importance
from sklearn.metrics import mean_squared_error
from scipy.stats import randint as sp_randint
from scipy.stats import randint as sp_uniform

# Carga de los df de features para entrenar y predecir

In [ ]:
windows = ["18_20","19_21","20_22","21_23"]
features = {}
label_auc = {}
label_inst = {}
label_clas_auc = {}
label_clas_inst = {}

for window in windows:
    features[window] = pd.read_csv("windows/{}/features.csv".format(window), index_col="ref_hash")
    label_auc[window] = pd.read_csv("windows/{}/labels_auc.csv".format(window), index_col="ref_hash")
    label_inst[window] = pd.read_csv("windows/{}/labels_inst.csv".format(window), index_col="ref_hash")
    label_clas_auc[window] = pd.DataFrame({'ref_hash': label_auc[window].index, 'label_auc': (label_auc[window]['label_auc'] == 259200).astype(int)}).set_index("ref_hash")
    label_clas_inst[window] = pd.DataFrame({'ref_hash': label_inst[window].index, 'label_inst': (label_inst[window]['label_inst'] == 259200).astype(int)}).set_index("ref_hash")
    
features_to_predict = pd.read_csv("windows/24_26/features.csv", index_col="ref_hash") 

## Selección de features

In [ ]:
def select_features(df, feature_list):
    return df.reindex(columns=feature_list)

In [ ]:
features_list_class_auc = [
    'appearances_in_auctions', 
    'user_appeared_last_day', 
    'time_to_reappear',
    #'amount_of_clicks', 
    #'has_installed', 
    #'user_clicked_last_day',
    #'user_installed_last_day', 
    #'amount_of_installs', 
    #'cant_max_day',
    #'cant_min_day', 
    #'mean_time_to_click', 
    #'max_time_click',
    #'min_time_click', 
    'mean_auctions_per_day', 
    #'mean_events_per_day',
    #'mean_clicks_per_day', 
    'amount_auctions_in_last_hour',
    'amount_auctions_in_last_2_hours', 
    'amount_auctions_in_last_5_hours',
    'amount_auctions_in_last_12_hours', 
    #'amount_auctions_in_last_24_hours',
    #'amount_events_in_last_hour', 
    #'amount_events_in_last_2_hours',
    #'amount_auctions_in_first_hour', 
    #'amount_auctions_in_first_3_hours',
    #'amount_auctions_in_first_5_hours', 
    'amount_auctions_in_first_12_hours',
    #'amount_events_in_first_hour', 
    #'amount_events_in_first_5_hours',
    'amount_events_in_first_12_hours', 
    #'amount_clicks_in_last_2_hours',
    #'amount_clicks_in_last_4_hours', 
    'device_os', 
    #'std_time_to_click',
    'std_time_auctions', 
    #'max_time_install', 
    #'min_time_install',
    #'mean_time_install', 
    #'std_time_install', 
    #'max_time_events',
    #'min_time_events', 
    #'mean_time_events', 
    'std_time_events',
    #'installs_per_events', 
    #'installs_per_clicks', 
    #'events_x_app_210',
    #'events_x_app_122', 
    #'events_x_app_65', 
    #'events_x_app_121',
    #'events_x_app_26', 
    #'most_installed_apps_used', 
    #'cant_apps_used',
    #'cant_events_atributed', 
    #'has_events_atributed',
    #'has_events_ids_with_installs', 
    #'has_events_ids_without_installs',
    'cant_events_0_4', 
    'cant_events_4_8', 
    'cant_events_8_12',
    'cant_events_12_16', 
    'cant_events_16_20', 
    'cant_events_20_24',
    'cant_auctions_0_4', 
    'cant_auctions_4_8', 
    'cant_auctions_8_12',
    'cant_auctions_12_16', 
    'cant_auctions_16_20', 
    'cant_auctions_20_24',
    #'implicit', 
    #'latitude', 
    #'longitude', 
    #'clicked_in_last_5_minutes',
    #'clicked_with_wifi_in_last_3_hours', 
    'hour_install', 
    'hour_events',
    'hour_clicks', 
    'hour_auctions',
    #'amount_auctions_in_last_half_hour',
    #'amount_auctions_in_last_15_minutes',
    "mean_appearances_events",
    "std_appearances_events",
    "std_appearances_auctions",
    'cant_devices',
    "mean_appearances_auctions",
    "mean_of_first_auction",
    "mean_of_first_event",
    "time_to_first_appearance_in_auctions",
    "mean_auctions_last_24hs",
    "mean_auctions_last_48hs",
    "std_auctions_last_24hs",
    "std_auctions_last_48hs", 
    #"mean_auctions_last_hour", 
    #"mean_auctions_last_4hs", 
    "mean_events_last_24hs", 
    "mean_events_last_48hs",  
    "std_events_last_24hs", 
    "std_events_last_48hs",
    #"mean_auctions_last_60hs",
    #"mean_auctions_last_36hs",
    #"mean_installs_last_24hs",
    #"mean_installs_last_48hs",
    "installs_per_auctions"
]

In [96]:
features_list_class_auc = [
    'appearances_in_auctions', 
    'user_appeared_last_day', 
    'time_to_reappear',
    'amount_of_clicks', 
    'has_installed', 
    'user_clicked_last_day',
    'user_installed_last_day', 
    'amount_of_installs', 
    'cant_max_day',
    'cant_min_day', 
    'mean_time_to_click', 
    'max_time_click',
    'min_time_click', 
    'mean_auctions_per_day', 
    'mean_events_per_day',
    'mean_clicks_per_day', 
    'amount_auctions_in_last_hour',
    'amount_auctions_in_last_2_hours', 
    'amount_auctions_in_last_5_hours',
    'amount_auctions_in_last_12_hours', 
    'amount_auctions_in_last_24_hours',
    'amount_events_in_last_hour', 
    'amount_events_in_last_2_hours',
    'amount_auctions_in_first_hour', 
    'amount_auctions_in_first_3_hours',
    'amount_auctions_in_first_5_hours', 
    'amount_auctions_in_first_12_hours',
    'amount_events_in_first_hour', 
    'amount_events_in_first_5_hours',
    'amount_events_in_first_12_hours', 
    'amount_clicks_in_last_2_hours',
    'amount_clicks_in_last_4_hours', 
    'device_os', 
    'std_time_to_click',
    'std_time_auctions', 
    'max_time_install', 
    'min_time_install',
    'mean_time_install', 
    'std_time_install', 
    'max_time_events',
    'min_time_events', 
    'mean_time_events', 
    'std_time_events',
    'installs_per_events', 
    'installs_per_clicks', 
    'events_x_app_210',
    'events_x_app_122', 
    'events_x_app_65', 
    'events_x_app_121',
    'events_x_app_26', 
    'most_installed_apps_used', 
    'cant_apps_used',
    'cant_events_atributed', 
    'has_events_atributed',
    'has_events_ids_with_installs', 
    'has_events_ids_without_installs',
    'cant_events_0_4', 
    'cant_events_4_8', 
    'cant_events_8_12',
    'cant_events_12_16', 
    'cant_events_16_20', 
    'cant_events_20_24',
    'cant_auctions_0_4', 
    'cant_auctions_4_8', 
    'cant_auctions_8_12',
    'cant_auctions_12_16', 
    'cant_auctions_16_20', 
    'cant_auctions_20_24',
    'implicit', 
    'latitude', 
    'longitude', 
    'clicked_in_last_5_minutes',
    'clicked_with_wifi_in_last_3_hours', 
    'hour_install', 
    'hour_events',
    'hour_clicks', 
    'hour_auctions',
    'amount_auctions_in_last_half_hour',
    'amount_auctions_in_last_15_minutes',
    "mean_appearances_events",
    "std_appearances_events",
    "std_appearances_auctions",
    'cant_devices',
    "mean_appearances_auctions",
    "mean_of_first_auction",
    "mean_of_first_event",
    "time_to_first_appearance_in_auctions",
    "mean_auctions_last_24hs",
    "mean_auctions_last_48hs",
    "std_auctions_last_24hs",
    "std_auctions_last_48hs", 
    "mean_auctions_last_hour", 
    "mean_auctions_last_4hs", 
    "mean_events_last_24hs", 
    "mean_events_last_48hs",  
    "std_events_last_24hs", 
    "std_events_last_48hs",
    "mean_auctions_last_60hs",
    "mean_auctions_last_36hs",
    "mean_installs_last_24hs",
    "mean_installs_last_48hs",
    "installs_per_auctions",
    "dayofweek_auctions",
    "dayofweek_installs"
]

In [ ]:
features_list_class_inst = [
    'appearances_in_auctions', 
    'user_appeared_last_day', 
    'time_to_reappear',
    #'amount_of_clicks', 
    'has_installed', 
    'user_clicked_last_day',
    'user_installed_last_day', 
    'amount_of_installs', 
    #'cant_max_day',
    #'cant_min_day', 
    #'mean_time_to_click', 
    #'max_time_click',
    #'min_time_click', 
    'mean_auctions_per_day', 
    'mean_events_per_day',
    #'mean_clicks_per_day', 
    'amount_auctions_in_last_hour',
    'amount_auctions_in_last_2_hours', 
    'amount_auctions_in_last_5_hours',
    'amount_auctions_in_last_12_hours', 
    'amount_auctions_in_last_24_hours',
    'amount_events_in_last_hour', 
    'amount_events_in_last_2_hours',
    'amount_auctions_in_first_hour', 
    'amount_auctions_in_first_3_hours',
    'amount_auctions_in_first_5_hours', 
    'amount_auctions_in_first_12_hours',
    'amount_events_in_first_hour', 
    'amount_events_in_first_5_hours',
    'amount_events_in_first_12_hours', 
    #'amount_clicks_in_last_2_hours',
    #'amount_clicks_in_last_4_hours', 
    'device_os', 
    #'std_time_to_click',
    'std_time_auctions', 
    #'max_time_install', 
    #'min_time_install',
    #'mean_time_install', 
    #'std_time_install', 
    #'max_time_events',
    #'min_time_events', 
    'mean_time_events', 
    'std_time_events',
    #'installs_per_events', 
    #'installs_per_clicks', 
    'events_x_app_210',
    'events_x_app_122', 
    'events_x_app_65', 
    'events_x_app_121',
    'events_x_app_26', 
    'most_installed_apps_used', 
    'cant_apps_used',
    'cant_events_atributed', 
    'has_events_atributed',
    #'has_events_ids_with_installs', 
    #'has_events_ids_without_installs',
    'cant_events_0_4', 
    'cant_events_4_8', 
    'cant_events_8_12',
    'cant_events_12_16', 
    'cant_events_16_20', 
    'cant_events_20_24',
    'cant_auctions_0_4', 
    'cant_auctions_4_8', 
    'cant_auctions_8_12',
    'cant_auctions_12_16', 
    'cant_auctions_16_20', 
    'cant_auctions_20_24',
    #'implicit', 
    #'latitude', 
    #'longitude', 
    #'clicked_in_last_5_minutes',
    'clicked_with_wifi_in_last_3_hours', 
    'hour_install', 
    'hour_events',
    'hour_clicks', 
    'hour_auctions',
    'cant_installs_0_4',
    'cant_installs_4_8', 
    'cant_installs_8_12',
    'cant_installs_12_16',
    'cant_installs_16_20',
    'cant_installs_20_24',
    "mean_appearances_clicks",
    "mean_appearances_installs",
    'amount_auctions_in_last_half_hour',
    'amount_auctions_in_last_15_minutes',
    'cant_devices', 
    "mean_appearances_auctions",
   "mean_appearances_events",
   "std_appearances_events",
    "std_appearances_auctions",
    "mean_of_first_auction",
    "mean_of_first_event",
    "time_to_first_appearance_in_auctions",
    "mean_auctions_last_24hs",
    "mean_auctions_last_48hs",
    "std_auctions_last_24hs",
    "std_auctions_last_48hs",
    "mean_auctions_last_hour", 
    "mean_auctions_last_4hs", 
    "mean_events_last_24hs", 
    "mean_events_last_48hs",  
    "std_events_last_24hs", 
    "std_events_last_48hs",
    "mean_auctions_last_60hs",
    "mean_auctions_last_36hs",
    "mean_installs_last_24hs",
    "mean_installs_last_48hs",
    "installs_per_auctions"
    
]

In [97]:
features_list_class_inst = [
    'appearances_in_auctions', 
    'user_appeared_last_day', 
    'time_to_reappear',
    'amount_of_clicks', 
    'has_installed', 
    'user_clicked_last_day',
    'user_installed_last_day', 
    'amount_of_installs', 
    'cant_max_day',
    'cant_min_day', 
    'mean_time_to_click', 
    'max_time_click',
    'min_time_click', 
    'mean_auctions_per_day', 
    'mean_events_per_day',
    'mean_clicks_per_day', 
    'amount_auctions_in_last_hour',
    'amount_auctions_in_last_2_hours', 
    'amount_auctions_in_last_5_hours',
    'amount_auctions_in_last_12_hours', 
    'amount_auctions_in_last_24_hours',
    'amount_events_in_last_hour', 
    'amount_events_in_last_2_hours',
    'amount_auctions_in_first_hour', 
    'amount_auctions_in_first_3_hours',
    'amount_auctions_in_first_5_hours', 
    'amount_auctions_in_first_12_hours',
    'amount_events_in_first_hour', 
    'amount_events_in_first_5_hours',
    'amount_events_in_first_12_hours', 
    'amount_clicks_in_last_2_hours',
    'amount_clicks_in_last_4_hours', 
    'device_os', 
    'std_time_to_click',
    'std_time_auctions', 
    'max_time_install', 
    'min_time_install',
    'mean_time_install', 
    'std_time_install', 
    'max_time_events',
    'min_time_events', 
    'mean_time_events', 
    'std_time_events',
    'installs_per_events', 
    'installs_per_clicks', 
    'events_x_app_210',
    'events_x_app_122', 
    'events_x_app_65', 
    'events_x_app_121',
    'events_x_app_26', 
    'most_installed_apps_used', 
    'cant_apps_used',
    'cant_events_atributed', 
    'has_events_atributed',
    'has_events_ids_with_installs', 
    'has_events_ids_without_installs',
    'cant_events_0_4', 
    'cant_events_4_8', 
    'cant_events_8_12',
    'cant_events_12_16', 
    'cant_events_16_20', 
    'cant_events_20_24',
    'cant_auctions_0_4', 
    'cant_auctions_4_8', 
    'cant_auctions_8_12',
    'cant_auctions_12_16', 
    'cant_auctions_16_20', 
    'cant_auctions_20_24',
    'implicit', 
    'latitude', 
    'longitude', 
    'clicked_in_last_5_minutes',
    'clicked_with_wifi_in_last_3_hours', 
    'hour_install', 
    'hour_events',
    'hour_clicks', 
    'hour_auctions',
    'cant_installs_0_4',
    'cant_installs_4_8', 
    'cant_installs_8_12',
    'cant_installs_12_16',
    'cant_installs_16_20',
    'cant_installs_20_24',
    "mean_appearances_clicks",
    "mean_appearances_installs",
    'amount_auctions_in_last_half_hour',
    'amount_auctions_in_last_15_minutes',
    'cant_devices', 
    "mean_appearances_auctions",
    "mean_appearances_events",
    "std_appearances_events",
    "std_appearances_auctions",
    "mean_of_first_auction",
    "mean_of_first_event",
    "time_to_first_appearance_in_auctions",
    "mean_auctions_last_24hs",
    "mean_auctions_last_48hs",
    "std_auctions_last_24hs",
    "std_auctions_last_48hs",
    "mean_auctions_last_hour", 
    "mean_auctions_last_4hs", 
    "mean_events_last_24hs", 
    "mean_events_last_48hs",  
    "std_events_last_24hs", 
    "std_events_last_48hs",
    "mean_auctions_last_60hs",
    "mean_auctions_last_36hs",
    "mean_installs_last_24hs",
    "mean_installs_last_48hs",
    "installs_per_auctions",
    "dayofweek_auctions",
    "dayofweek_installs"
]

## Funciones útiles para el entrenamento y el balanceo

In [ ]:
def get_df_to_train(windows, features, labels, feature_list):
    df_list = []
    for window in windows:
        df = select_features(features[window], feature_list).join(labels[window], how="inner")
        df_list.append(df)
    df_full = pd.concat(df_list)
    df_full.reset_index(inplace=True, drop=True)
    return df_full

In [ ]:
# factor = 1 100% balanced
# factor = 0 not balanced
# label_value label que mas aparece y se debe balancear
def balance(df_full, label_name, factor, label_value = 1):
    cant_values = df_full[label_name].value_counts()[label_value]
    cant_no_values = len(df_full[label_name]) - cant_values
    a_borrar = int((cant_values-cant_no_values)*factor)
    index_to_drop = np.random.choice(df_full.loc[df_full[label_name] == label_value].index,a_borrar, replace=False)
    return df_full.drop(index=index_to_drop)

In [ ]:
def get_train_test_split(df_full, label_name, test_size=0.3):
    y = df_full[label_name]
    X_data = df_full.drop(label_name, axis=1)
    return train_test_split(X_data, y, test_size=test_size, random_state=7)

# Importancia de los features

In [ ]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor

> ### ***Importancia de los features de auctions***

In [ ]:
df_auctions_importance = get_df_to_train(windows, features, label_auc, features_list_class_auc)
df_auctions_importance = df_auctions_importance.sample(10000)


df_auctions_features = df_auctions_importance.drop('label_auc', axis=1)
df_auctions_labels = df_auctions_importance['label_auc']

X = df_auctions_features.fillna(0)
Y = df_auctions_labels
names = X.columns.tolist()
rf_auc = RandomForestRegressor(n_estimators=100)
rf_auc.fit(X, Y)
#print("Features sorted by their score:")
#print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), reverse=True))

In [ ]:
feature_importance_auctions = sorted(zip(map(lambda x: round(x, 4), rf_auc.feature_importances_), names))
feature_auctions_plot = pd.DataFrame(feature_importance_auctions).set_index(1)

In [ ]:
# Ploteo

import matplotlib.pyplot as plt

plot = feature_auctions_plot.plot(kind="barh", figsize=(14,18))
plot.set_title("Importance of auctions features", fontsize=25)
plot.set_ylabel("Feature", fontsize=20)
plot.set_xlabel("Importance", fontsize=20);
plot.legend(["Importance"], loc=4)
plot.tick_params(labelsize=13, which='major')
plt.savefig("importance_auctions.png", bbox_inches="tight")

> ### ***Importancia de los features de installs***

In [ ]:
df_installs_importance = get_df_to_train(windows, features, label_inst, features_list_class_inst)
df_installs_importance = df_installs_importance.sample(50000)


df_installs_features = df_installs_importance.drop('label_inst', axis=1)
df_installs_labels = df_installs_importance['label_inst']

X = df_installs_features.fillna(0)
Y = df_installs_labels
names = X.columns.tolist()
rf_inst = RandomForestRegressor(n_estimators=100)
rf_inst.fit(X, Y)
#print("Features sorted by their score:")
#print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), reverse=True))

In [ ]:
feature_importance_inst = sorted(zip(map(lambda x: round(x, 4), rf_inst.feature_importances_), names))
feature_inst_plot = pd.DataFrame(feature_importance_inst).set_index(1)

In [ ]:
# Ploteo

import matplotlib.pyplot as plt

plot = feature_inst_plot.plot(kind="barh", figsize=(14,18))
plot.set_title("Importance of installs features", fontsize=25)
plot.set_ylabel("Feature", fontsize=20)
plot.set_xlabel("Importance", fontsize=20);
plot.legend(["Importance"], loc=4)
plot.tick_params(labelsize=13, which='major')
plt.savefig("importance_installs.png", bbox_inches="tight")

# ***Modelos***

# **Redes neuronales**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics

> ### ***Modelo de redes neuronales de predicción de auctions***

In [ ]:
df_full_auctions = get_df_to_train(windows, features, label_auc, features_list_class_auc)
df_full_auctions = df_full_auctions.fillna(0)

# Filtro los tiempos máximos.
df_full_auctions = df_full_auctions[df_full_auctions["label_auc"] < 259200]

# Separo train y test.
X_train_auc, X_test_auc, y_train_auc, y_test_auc = get_train_test_split(df_full_auctions, "label_auc")

In [ ]:
# Creo el modelo
model_auc_neural = Sequential()

# Capa de entrada
model_auc_neural.add(Dense(10, input_dim=X_train_auc.shape[1], activation='relu'))
# Dropout para controlar el overfit
model_auc_neural.add(Dropout(0.2))

# Capa con 40 neuronas
model_auc_neural.add(Dense(20, activation='relu'))
# Dropout para controlar el overfit
#model_auc_neural.add(Dropout(0.2))

# Capa con 60 neuronas
model_auc_neural.add(Dense(40, activation='relu'))
# Dropout para controlar el overfit
#model_auc_neural.add(Dropout(0.2))

# Salida del modelo
model_auc_neural.add(Dense(1))

# Compilo el modelo
model_auc_neural.compile(optimizer ='adam', loss = 'mean_squared_error', metrics =[metrics.mse])

In [ ]:
# Veo el modelo
model_auc_neural.summary()

In [ ]:
# Entreno el modelo
model_auc_neural.fit(X_train_auc, y_train_auc, validation_data=(X_test_auc,y_test_auc), epochs=5, batch_size=32)

In [ ]:
# Acá pones el valor que da el "mean_squared_error" del modelo
MSE = 4360503664.0435

print(f"RMSE: {MSE**(1/2)}")

> ### ***Modelo de redes neuronales de predicción de installs***

In [ ]:
df_full_inst = get_df_to_train(windows, features, label_inst, features_list_class_inst)
df_full_inst = df_full_inst.fillna(0)

# Saco los tiempos máximos

df_full_inst = df_full_inst[df_full_inst["label_inst"] < 259200]
#df_full_inst = balance(df_full_inst, "label_inst", 1, 259200)


# Separo en train y test
X_train_inst, X_test_inst, y_train_inst, y_test_inst = get_train_test_split(df_full_inst, "label_inst")

In [ ]:
# Creo el modelo
model_inst_neural = Sequential()

# Capa de entrada
model_inst_neural.add(Dense(10, input_dim=X_train_inst.shape[1], activation='relu'))
# Dropout para controlar el overfit


# Capa con 30 neuronas
model_inst_neural.add(Dense(20, activation='relu'))
# Dropout para controlar el overfit

# Capa con 50 neuronas
model_inst_neural.add(Dense(40, activation='relu'))
model_inst_neural.add(Dense(80, activation='relu'))
model_inst_neural.add(Dense(120, activation='relu'))
model_inst_neural.add(Dense(160, activation='relu'))
# Dropout para controlar el overfit

# Salida del modelo
model_inst_neural.add(Dense(1))

# Compilo el modelo
model_inst_neural.compile(optimizer ='adam', loss = 'mean_squared_error', metrics =[metrics.mse])

In [ ]:
# Veo el modelo
model_inst_neural.summary()

In [ ]:
# Entreno el modelo
model_inst_neural.fit(X_train_inst, y_train_inst, validation_data=(X_test_inst,y_test_inst), epochs=50)

In [ ]:
# Acá pones el valor que da el "mean_squared_error" del modelo
MSE = 6272147912.2780

print(f"RMSE: {MSE**(1/2)}")

# **XGBoost**

> ### ***Modelo de XGBoost de predicción de auctions***

In [ ]:
df_full = get_df_to_train(windows, features, label_auc, features_list_class_auc)

#df_full = df_full.sample(int(len(df_full)))

#df_full = balance(df_full, "label_auc", 0, 259200)

df_full = df_full.loc[df_full["label_auc"] != 259200]

X_train, X_test, y_train, y_test = get_train_test_split(df_full, "label_auc", 0.2)

In [ ]:
model_auc = xgb.XGBRegressor(   
    gamma=0.1, 
    learning_rate=0.1, 
    max_depth=3,  
    n_estimators=500,
    n_jobs=2,  
    objective='reg:linear',   
    silent=True
)

In [ ]:
%%time
model_auc.fit(X_train, y_train, eval_metric='rmse')
#cv = xgb.cv(params, xg_train, 5000, nfold=n_folds, early_stopping_rounds=early_stopping, verbose_eval=1)
prediction = model_auc.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, prediction))
print("RMSE: %f" % (rmse))

In [ ]:
model_auc.save_model('xgboost_auc.model')

> #### ***Mejor modelo de auctions***

RMSE: 63345.641573
CPU times: user 25min 48s, sys: 8.65 s, total: 25min 57s
Wall time: 26min 8s
    
model_auc = xgb.XGBRegressor(
    base_score=0.5, 
    booster='gbtree', 
    colsample_bylevel=1,
    colsample_bytree=1, 
    gamma=0.1, 
    learning_rate=0.1, 
    max_delta_step=0,
    max_depth=4, 
    min_child_weight=1, 
    missing=None, 
    n_estimators=600,
    n_jobs=1, 
    nthread=None, 
    objective='reg:linear', 
    random_state=0,
    reg_alpha=0, 
    reg_lambda=1, 
    scale_pos_weight=1, 
    seed=None,
    silent=True, 
    subsample=1
)

In [ ]:
_ = plot_importance(model_auc, height=0.5)

In [ ]:
from xgboost import plot_importance, plot_tree
_ = plot_importance(model_auc, height=0.5)

> ### ***Modelo de XGBoost de predicción de installs***

In [ ]:
df_full_installs = get_df_to_train(windows, features, label_inst, features_list_class_inst)

df_full_installs = df_full_installs.sample(int(len(df_full_installs)))

#df_full_installs = balance(df_full_installs, "label_inst", 1, 259200)
df_full_installs = df_full_installs[df_full_installs["label_inst"] != 259200]

X_train, X_test, y_train, y_test = get_train_test_split(df_full_installs, "label_inst", 0.2)

In [ ]:
model_inst = xgb.XGBRegressor(
    learning_rate = 0.1,
    n_estimators=700,
    max_dept=6,
    gamma=0.1,
    n_jobs=-1
)

In [ ]:
%%time
model_inst.fit(X_train, y_train, eval_metric='rmse', verbose=True)

prediction = model_inst.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, prediction))
print("RMSE: %f" % (rmse))

In [ ]:
model_inst.save_model('xgboost_inst.model')

In [ ]:
from xgboost import plot_importance, plot_tree
_ = plot_importance(model_inst, height=0.9)

> #### ***Mejor modelo de installs***

WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 76768.119164
CPU times: user 5min 39s, sys: 2 s, total: 5min 41s
Wall time: 5min 44s

model_inst = xgb.XGBRegressor(
    learning_rate = 0.1,
    n_estimators=700,
    max_dept=5,
    gamma=0.1,
    n_jobs=-1
)"

# **LightGBM**

### LightGBM para auctions

In [98]:
df_full_auctions = get_df_to_train(windows, features, label_auc, features_list_class_auc)
df_full_auctions = df_full_auctions[df_full_auctions["label_auc"] != 259200]
#df_full_auctions = df_full_auctions.sample()

In [99]:
X_train_auc, X_test_auc, y_train_auc, y_test_auc = get_train_test_split(df_full_auctions, "label_auc", 0.2)

In [100]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train_auc, y_train_auc)
lgb_eval = lgb.Dataset(X_test_auc, y_test_auc, reference=lgb_train)

params = {
    'n_estimators' : 500,
    'boosting_type': 'gbdt',
    #'boosting_type': 'dart',
    'objective': 'regression',
    'metric': {'l1', 'l2'},
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 326,
    'min_child_weight': 10000.0,
    'reg_alpha': 50,
    'reg_lambda': 0,
    'verbose': 0
}

gbm_auc = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval)
                #early_stopping_rounds=100)



pred_auc_gbm = gbm_auc.predict(X_test_auc, num_iteration=gbm_auc.best_iteration)
print('The rmse of prediction is:', mean_squared_error(y_test_auc, pred_auc_gbm) ** 0.5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 4.80641e+09	valid_0's l1: 56388.5
[2]	valid_0's l2: 4.7355e+09	valid_0's l1: 55917.2
[3]	valid_0's l2: 4.67114e+09	valid_0's l1: 55481.7
[4]	valid_0's l2: 4.61294e+09	valid_0's l1: 55080.3
[5]	valid_0's l2: 4.56018e+09	valid_0's l1: 54709.7
[6]	valid_0's l2: 4.51237e+09	valid_0's l1: 54367.2
[7]	valid_0's l2: 4.46912e+09	valid_0's l1: 54050.7
[8]	valid_0's l2: 4.4298e+09	valid_0's l1: 53757.9
[9]	valid_0's l2: 4.39403e+09	valid_0's l1: 53486.9
[10]	valid_0's l2: 4.36161e+09	valid_0's l1: 53235.7
[11]	valid_0's l2: 4.33221e+09	valid_0's l1: 53002.3
[12]	valid_0's l2: 4.30557e+09	valid_0's l1: 52788.4
[13]	valid_0's l2: 4.28114e+09	valid_0's l1: 52588.4
[14]	valid_0's l2: 4.25896e+09	valid_0's l1: 52402.2
[15]	valid_0's l2: 4.23899e+09	valid_0's l1: 52230.4
[16]	valid_0's l2: 4.22078e+09	valid_0's l1: 52070.7
[17]	valid_0's l2: 4.20388e+09	valid_0's l1: 51919
[18]	valid_0's l2: 4.18832e+09	valid_0's l1: 51775.3
[19]	valid_0's l2: 4.17457e+09	valid_0's l1: 51646
[20]	val

In [ ]:
63315.97962183445

In [ ]:
gbm_auc.save_model('model_light_gbm_auc.txt')

In [ ]:
_ = gbm_auc.plot_importance(model)

### LightGBM para installs

In [93]:
df_full_installs = get_df_to_train(windows, features, label_inst, features_list_class_inst)
df_full_installs = df_full_installs[df_full_installs["label_inst"] != 259200]

In [94]:
X_train_inst, X_test_inst, y_train_inst, y_test_inst = get_train_test_split(df_full_installs, "label_inst", 0.05)

In [95]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train_inst, y_train_inst)
lgb_eval = lgb.Dataset(X_test_inst, y_test_inst, reference=lgb_train)

params = {
    'n_estimators' : 1000,
    'boosting_type': 'gbdt',
    #'boosting_type': 'dart',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 6,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 185,
    'min_child_weight': 100.0,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'verbose': 0
}

gbm_inst = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval)
                #early_stopping_rounds=80)

pred_inst_gbm = gbm_inst.predict(X_test_inst, num_iteration=gbm_inst.best_iteration)
print('The rmse of prediction is:', mean_squared_error(y_test_inst, pred_inst_gbm) ** 0.5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 5.93669e+09	valid_0's l1: 67501.3
[2]	valid_0's l2: 5.93505e+09	valid_0's l1: 67491.8
[3]	valid_0's l2: 5.93367e+09	valid_0's l1: 67483.5
[4]	valid_0's l2: 5.93221e+09	valid_0's l1: 67474
[5]	valid_0's l2: 5.93095e+09	valid_0's l1: 67465.6
[6]	valid_0's l2: 5.92957e+09	valid_0's l1: 67456.2
[7]	valid_0's l2: 5.92846e+09	valid_0's l1: 67448.9
[8]	valid_0's l2: 5.92745e+09	valid_0's l1: 67441.5
[9]	valid_0's l2: 5.92607e+09	valid_0's l1: 67432.2
[10]	valid_0's l2: 5.92527e+09	valid_0's l1: 67425.7
[11]	valid_0's l2: 5.92464e+09	valid_0's l1: 67420.6
[12]	valid_0's l2: 5.92406e+09	valid_0's l1: 67415.8
[13]	valid_0's l2: 5.92326e+09	valid_0's l1: 67409.8
[14]	valid_0's l2: 5.92285e+09	valid_0's l1: 67405.9
[15]	valid_0's l2: 5.92208e+09	valid_0's l1: 67400.2
[16]	valid_0's l2: 5.92132e+09	valid_0's l1: 67394.5
[17]	valid_0's l2: 5.92072e+09	valid_0's l1: 67389.4
[18]	valid_0's l2: 5.92014e+09	valid_0's l1: 67384.2
[19]	valid_0's l2: 5.9195e+09	valid_0's l1: 67378.8
[20]	

In [ ]:
76847.61354285045

In [ ]:
gbm_inst.save_model('model_light_gbm_inst.txt')

In [ ]:
pred_inst_gbm

## Tuneo de hiperparámetros para LightGBM

In [ ]:
gridParams = {  #para el grid search
    'learning_rate': [0.05, 0.1, 0.5],
    'n_estimators': [1000],
    'num_leaves': [5,10,15,20],
    'boosting_type' : ['gbdt', 'dart'],
    'objective' : ['regression'],
    'metric' : ['l2', 'l1'],
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.3,0.6],
    'reg_alpha' : [1,2],
    'reg_lambda' : [1,1.5,2],
    }


In [ ]:
param_test ={'num_leaves': sp_randint(6, 50), #para el randomized search
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

> ## Tuneo para el modelo de installs

>> #### Randomized Search

In [ ]:
clf = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='l2', n_jobs=4, n_estimators = 500)
gs_inst = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    scoring='neg_mean_squared_error',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
%%time
gs_inst.fit(X_train_inst, y_train_inst)
print('Best score reached: {} with params: {} '.format(np.sqrt(gs_inst.best_score_*(-1)), gs_inst.best_params_))

>> #### Grid Search

In [ ]:
lgm_grid_inst = lgb.LGBMRegressor(max_depth=-1, learning_rate = 0.01, random_state=314, silent=True, metric='l2', n_jobs=4, n_estimators = 500)

gsearchlgm = GridSearchCV(estimator = lgm_grid_inst, param_grid = gridParams,\
                          scoring= 'neg_mean_squared_error', n_jobs=4,iid=False, cv=4)
gsearchlgm.fit(df_full_installs[features_list_class_inst], df_full_installs['label_inst'])
print('Best score reached: {} with params: {} '.format(np.sqrt(gsearchlgm.best_score_*(-1)), gsearchlgm.best_params_))

In [ ]:
print('Best score reached: {} with params: {} '.format(np.sqrt(gsearchlgm.best_score_*(-1)), gsearchlgm.best_params_))

> ## Tuneo para el modelo de auctions

>> #### Randomized Search

In [ ]:
clf_auc = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='l2', n_jobs=4, n_estimators = 500)
gs_auc = RandomizedSearchCV(
    estimator=clf_auc, param_distributions=param_test, 
    scoring='neg_mean_squared_error',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
df_full_auctions = df_full_auctions.sample(160000)
X_train_auc, X_test_auc, y_train_auc, y_test_auc = get_train_test_split(df_full_auctions, "label_auc")

In [ ]:
%%time
gs_auc.fit(X_train_auc, y_train_auc)
print('Best score reached: {} with params: {} '.format(np.sqrt(gs_auc.best_score_*(-1)), gs_auc.best_params_))

>> #### Grid Search

In [ ]:
df_full_auctions = df_full_auctions.sample(100000)

In [ ]:
lgm_grid_auc = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='l2', n_jobs=4, n_estimators = 500)

gsearchlgm_auc = GridSearchCV(estimator = lgm_grid_auc, param_grid = gridParams,\
                          scoring= 'neg_mean_squared_error', n_jobs=4,iid=False, cv=4)
gsearchlgm_auc.fit(df_full_auctions[features_list_class_auc],df_full_auctions['label_auc'])
print('Best score reached: {} with params: {} '.format(np.sqrt(gsearchlgm_auc.best_score_*(-1)), gsearchlgm_auc.best_params_))

# **Random Forest**

## Random Forest para auctions

In [ ]:
df_full_auctions = get_df_to_train(windows, features, label_auc, features_list_class_auc)
df_full_auctions = df_full_auctions[df_full_auctions["label_auc"] != 259200]

In [ ]:
X_train_auc, X_test_auc, y_train_auc, y_test_auc = get_train_test_split(df_full_auctions, "label_auc")

In [ ]:
random_forest_auc = RandomForestRegressor(
                           n_estimators=100, 
                           min_samples_split = 10, 
                           random_state=100
                         )

#random forest no maneja nulos
X_train_auc = X_train_auc.fillna(0)
X_test_auc = X_test_auc.fillna(0) 

random_forest_auc.fit(X_train_auc, y_train_auc.values.ravel())

params = {'objective': 'reg:squarederror',
          'eta': 0.3,
          'max_depth': 5,
          'min_child_weight': 3,
          'silent': 1,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'seed': 1}

num_trees=250

prediction_auc = random_forest_auc.predict(X_test_auc)

rmse = np.sqrt(mean_squared_error(y_test_auc, prediction_auc))
print("RMSE: %f" % (rmse))

## Random Forest para installs

In [ ]:
df_full_inst = get_df_to_train(windows, features, label_inst, features_list_class_inst)
df_full_inst = df_full_inst[df_full_inst["label_inst"] != 259200]

In [ ]:
X_train_inst, X_test_inst, y_train_inst, y_test_inst = get_train_test_split(df_full_inst, "label_inst")

In [ ]:
random_forest_inst = RandomForestRegressor(
                           n_estimators=1000, 
                           min_samples_split = 10, 
                           random_state=100
                         )

#random forest no maneja nulos
X_train_inst = X_train_inst.fillna(0)
X_test_inst = X_test_inst.fillna(0) 

random_forest_inst.fit(X_train_inst, y_train_inst.values.ravel())

params = {'objective': 'reg:squarederror',
          'eta': 0.3,
          'max_depth': 5,
          'min_child_weight': 3,
          'silent': 1,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'seed': 1}

num_trees=250

prediction_inst = random_forest_inst.predict(X_test_inst)

rmse = np.sqrt(mean_squared_error(y_test_inst, prediction_inst))
print("RMSE: %f" % (rmse))

# Predecir

### Predicción auctions XGBoost

In [ ]:
pred_auctions = model_auc.predict(select_features(features_to_predict, features_list_class_auc))
df_preds_auctions = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : pred_auctions})
df_preds_auctions.to_csv("auctions_predictions_xgboost.csv", index=False) #persistimos los resultados

### Predicción auctions Redes Neuronales

In [ ]:
pred_auctions_neural = model_auc_neural.predict(select_features(features_to_predict, features_list_class_auc).fillna(0))
pred_auctions_neural = np.array(pred_auctions_neural).flatten()
df_preds_auctions_neural = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : pred_auctions_neural})
df_preds_auctions_neural.to_csv("auctions_predictions_neural.csv", index=False) #persistimos los resultados

### Predicción auctions LightGBM

In [ ]:
pred_auc_gbm = gbm_auc.predict(select_features(features_to_predict,features_list_class_auc),\
                               num_iteration=gbm_auc.best_iteration)
df_preds_auc_gbm = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : pred_auc_gbm})
df_preds_auc_gbm.to_csv("auctions_predictions_lightgbm.csv", index=False) #persistimos los resultados

### Predicción auctions Random Forests

In [ ]:
preds_auc_rf = random_forest_auc.predict(select_features(features_to_predict,features_list_class_auc).fillna(0))
df_preds_auc_rf = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : preds_auc_rf})
df_preds_auc_rf.to_csv("auctions_predictions_randomForest.csv", index=False) #persistimos los resultados

### Predicción installs XGBoost

In [ ]:
pred_installs = model_inst.predict(select_features(features_to_predict, features_list_class_inst))
df_preds_installs = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : pred_installs})
df_preds_installs.to_csv("installs_predictions_xgboost.csv", index=False) #persistimos los resultados

### Predicción installs Redes Neuronales

In [ ]:
pred_installs_neural = model_inst_neural.predict(select_features(features_to_predict, features_list_class_inst).fillna(0))
pred_installs_neural = np.array(pred_installs_neural).flatten()
df_preds_installs_neural = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : pred_installs_neural})
df_preds_installs_neural.to_csv("installs_predictions_neural.csv", index=False) #persistimos los resultados

### Predicción installs LightGBM

In [ ]:
pred_inst_gbm = gbm_inst.predict(select_features(features_to_predict,features_list_class_inst),\
                               num_iteration=gbm_inst.best_iteration)
df_preds_inst_gbm = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : pred_inst_gbm})
df_preds_inst_gbm.to_csv("installs_predictions_lightgbm.csv", index=False) #persistimos los resultados

### Predicción installs Random Forests

In [ ]:
preds_inst_rf = random_forest_inst.predict(select_features(features_to_predict,features_list_class_inst).fillna(0))
df_preds_inst_rf = pd.DataFrame({'ref_hash' : features_to_predict.index, 'obj' : preds_inst_rf})
df_preds_inst_rf.to_csv("installs_predictions_randomForest.csv", index=False) #persistimos los resultados

# Submit to Kaggle

In [ ]:
def export_df(df, name):
    df.to_csv(name, index=False)

Las predicciones tendrán seteadas como índice los ref_hash para no perder la referencia
No es necesario filtrar los ref_hash y quedarnos solo con los target en las predicciones que obtenemos ya que de eso
se encarga la función 

In [ ]:
target = pd.read_csv("target_competencia_ids.csv")

In [ ]:
#df_preds_installs = pd.read_csv("installs_predictions.csv")
#df_preds_auctions = pd.read_csv("auctions_predictions.csv")

In [ ]:
def create_submit_df(auctions_predictions, installs_predictions, target):
    
    target = target.set_index('ref_hash')
    
    auc = auctions_predictions.reset_index()
    auc.columns = ['ref_hash','obj']
    auc['ref_hash'] = auc['ref_hash'].astype(str) + "_st"
    auc = auc.set_index('ref_hash')
    
    ins = installs_predictions.reset_index()
    ins.columns = ['ref_hash','obj']
    ins['ref_hash'] = ins['ref_hash'].astype(str) + "_sc"
    ins = ins.set_index('ref_hash')
    
    frames = [ins,auc]
    submit_result = pd.concat(frames).reset_index()
    target_list = target.reset_index('ref_hash')['ref_hash'].tolist()
    return submit_result.loc[submit_result['ref_hash'].isin(target_list)].sort_values(by = 'ref_hash')

### XGBoost

In [ ]:
kaggle_sub = create_submit_df(df_preds_auctions.set_index('ref_hash'), \
                              df_preds_installs.set_index('ref_hash'), target)

In [ ]:
export_df(kaggle_sub, "submit.csv")

### Neural network

In [ ]:
kaggle_sub_neural = create_submit_df(df_preds_auctions_neural.set_index('ref_hash'), \
                              df_preds_installs_neural.set_index('ref_hash'), target)

In [ ]:
export_df(kaggle_sub_neural, "submit_neural.csv")

In [ ]:
ensamble_df = kaggle_sub.merge(kaggle_sub_neural, on="ref_hash")


In [ ]:
ensamble_df["obj"] = (ensamble_df["obj_x"] + ensamble_df["obj_y"])/2

In [ ]:
ensamble_kaggle = ensamble_df[["ref_hash", "obj"]]

In [ ]:
export_df(ensamble_kaggle, "submit_ensamble.csv")

#### Ensemble XGBoost + LightGBM

In [ ]:
kaggle_sub_lgbm = create_submit_df(df_preds_auc_gbm.set_index('ref_hash'), df_preds_inst_gbm.set_index('ref_hash'), target)
kaggle_sub_xgboost = create_submit_df(df_preds_auctions.set_index('ref_hash'), df_preds_installs.set_index('ref_hash'), target)

In [ ]:
xgboost_submit = kaggle_sub_lgbm.merge(kaggle_sub_xgboost, on = 'ref_hash')

In [ ]:
xgboost_submit['obj'] = (xgboost_submit['obj_x'] + xgboost_submit['obj_y'])/2

In [ ]:
xgboost_submit = xgboost_submit[['ref_hash', 'obj']]

In [ ]:
export_df(xgboost_submit, "submit_leo.csv")

### Ensamble lgbm + random_forests + xgboost

In [ ]:
a_inst = pd.read_csv("installs_predictions_lightgbm.csv")
b_inst = pd.read_csv("installs_predictions_randomForest.csv")
c_inst = pd.read_csv("installs_predictions_xgboost.csv")

In [ ]:
a_auc = pd.read_csv("auctions_predictions_lightgbm.csv")
b_auc = pd.read_csv("auctions_predictions_randomForest.csv")
c_auc = pd.read_csv("auctions_predictions_xgboost.csv")

In [ ]:
a_b_inst = a_inst.merge(b_inst, on="ref_hash")
all_inst = a_b_inst.merge(c_inst, on = "ref_hash")
all_inst['obj'] = (all_inst['obj_x'] + all_inst['obj_y'] + all_inst['obj'])/3
all_inst = all_inst[['ref_hash', 'obj']]

In [ ]:
a_b_auc = a_auc.merge(b_auc, on="ref_hash")
all_auc = a_b_auc.merge(c_auc, on = "ref_hash")
all_auc['obj'] = (all_auc['obj_x'] + all_auc['obj_y'] + all_auc['obj'])/3
all_auc = all_auc[['ref_hash', 'obj']]

In [ ]:
submit = create_submit_df(all_auc.set_index('ref_hash'), all_inst.set_index('ref_hash'), target)

In [ ]:
export_df(submit, "submit.csv")

### Submit con LightGBM

In [ ]:
export_df(create_submit_df(df_preds_auc_gbm.set_index('ref_hash'), df_preds_inst_gbm.set_index('ref_hash'), target), "submit_prueba.csv")